# Tangelo hands-on: VQE framework

## Before you jump in

Before tackling this hands-on, we suggest to have a look at:

- the tutorial on the basics of the linq module
- the tutorial on the basics of quantum chemistry modeling

This hands-on covers some material described in the [following tutorials](https://github.com/goodchemistryco/Tangelo-Examples/tree/main/examples/variational_methods).

You can use the cells below to install or import useful packages. We recommend to install qulacs and qiskit.

In [1]:
import numpy as np
import time

If you wish to install anything in your jupyter environment while working on this notebook, including Tangelo, you can go through your terminal and reload the upyter kernel or run `pip` commands directly from this notebook using `!`

In [2]:
# Uncomment and change package name to install it in the current environment
# !pip install tangelo-gc --quiet
# !pip install qulacs --quiet
# !pip install qiskit --quiet

## Level 1: VQE basics

Let's define a molecule in second quantization, for the sake of exploring the variational framework. Here we define a molecule explicitly, specifying a geometry, charge, spin and basis set, but you could also quickly just import other existing ones in our ["molecule library"](https://github.com/goodchemistryco/Tangelo/blob/main/tangelo/molecule_library.py) if you want to swap between other examples.

In [3]:
# Define H2 molecule in minimal basis set
from tangelo import SecondQuantizedMolecule

xyz_H2 = [("H", (0., 0., 0.)), ("H", (0., 0., 0.7414))]
mol_H2_sto3g = SecondQuantizedMolecule(xyz_H2, q=0, spin=0, basis="sto-3g")

mol = mol_H2_sto3g

### 1a)  Computing the ground state energy with VQE

Write a code cell that instantiates a `VQESolver` object and computes the ground state energy of our molecule. You can use the default options. You should find that the result for the optimal geometry of H2 in sto-3g basis is roughly -1.1372702 Ha.

In [4]:
# TODO: Instantiate VQE Solver, compute ground state energy with VQE

from tangelo.algorithms import VQESolver

vqe_options = {"molecule": mol}
vqe_solver = VQESolver(vqe_options)

vqe_solver.build()

opt_energy = vqe_solver.simulate()
print(opt_energy)

-1.1372701683419209


Many Can you find what qubit mapping, ansatz and classical optimizer were used in your code ?

In [5]:
# TODO : print that information
print(vars(vqe_solver))

{'molecule': SecondQuantizedMolecule(xyz=[('H', (0.0, 0.0, 0.0)), ('H', (0.0, 0.0, 0.7413999999999997))], q=0, spin=0, n_atoms=2, n_electrons=2, basis='sto-3g', ecp={}, symmetry=False, uhf=False, mf_energy=-1.1166843870853411, mo_energies=array([-0.57797481,  0.66969867]), mo_occ=array([2., 0.]), mean_field=RHF object of <class 'pyscf.scf.hf.RHF'>, n_mos=2, n_sos=4, active_occupied=[0], frozen_occupied=[], active_virtual=[1], frozen_virtual=[]), 'qubit_mapping': 'jw', 'ansatz': <tangelo.toolboxes.ansatz_generator.uccsd.UCCSD object at 0x1059bc1f0>, 'optimizer': <bound method VQESolver._default_optimizer of <tangelo.algorithms.variational.vqe_solver.VQESolver object at 0x16a77b580>>, 'initial_var_params': [2e-05, 0.036325371102345044], 'backend_options': {'target': None, 'n_shots': None, 'noise_model': None}, 'penalty_terms': None, 'deflation_circuits': [], 'deflation_coeff': 1, 'ansatz_options': {}, 'up_then_down': False, 'qubit_hamiltonian': (-0.0988639693354576+0j) [] +
(-0.045322202

### 1b) A closer look at what happened

Please print:

- the accuracy of the previous computation (difference between your optimal VQE energy and the result coming from the `FCISolver`)
- the optimal circuit, and the optimal parameters
- the qubit operator that encoded your molecule 

In [6]:
from tangelo.algorithms import FCISolver

fci_solver = FCISolver(mol)
fci_energy = fci_solver.simulate()

print(f"Accuracy :: {abs(opt_energy - fci_energy)}\n")
print(f"Optimal circuit ::\n {vqe_solver.optimal_circuit}")
print(f"Optimal parameters ::\n {vqe_solver.optimal_var_params}")

Accuracy :: 6.318983336583983e-09

Optimal circuit ::
 Circuit object. Size 158 

X         target : [0]   
X         target : [1]   
RX        target : [0]   parameter : 1.5707963267948966
H         target : [2]   
CNOT      target : [1]   control : [0]   
CNOT      target : [2]   control : [1]   
RZ        target : [2]   parameter : 12.566316203843638	 (variational)
CNOT      target : [2]   control : [1]   
CNOT      target : [1]   control : [0]   
H         target : [2]   
RX        target : [0]   parameter : -1.5707963267948966
H         target : [0]   
RX        target : [2]   parameter : 1.5707963267948966
CNOT      target : [1]   control : [0]   
CNOT      target : [2]   control : [1]   
RZ        target : [2]   parameter : 5.441051553448883e-05	 (variational)
CNOT      target : [2]   control : [1]   
CNOT      target : [1]   control : [0]   
RX        target : [2]   parameter : -1.5707963267948966
H         target : [0]   
RX        target : [1]   parameter : 1.5707963267948966

## Level 2: Impact of some of the options in VQE

A number of options are available in the VQE framework. Let's see how these choices impact the calculations.

### 2a) Qubit mappings

Our VQE framework supports a number of qubit mappings, such as Jordan-Wigner (`'jw'`), Bravyi-Kitaev (`'bk'`) or Symmetry-Conserving Bravyi-Kitaev (`'bk'`). We also support more uncommon ones, such as JKMN or HCB, and probably more in the future !

Can you instantiate `VQESolver` objects using these different qubit mappings, and see how they impact the complexity of the algorithm ?

In [7]:
for qb_mapping in ["JW", "BK", "SCBK", "JKMN"]:
    # TODO: Try different qubit mappings. How do they impact to the computational resource requirements ?
    vqe_options = {"molecule": mol_H2_sto3g, "qubit_mapping": qb_mapping}
    vqe_solver = VQESolver(vqe_options)
    vqe_solver.build()
    print(vqe_solver.get_resources())    
    opt_energy = vqe_solver.simulate()
    print(f"{qb_mapping} :: {opt_energy:=}\n{vqe_solver.qubit_hamiltonian}\n")
    print(vqe_solver.get_resources())

{'qubit_hamiltonian_terms': 15, 'circuit_width': 4, 'circuit_depth': 100, 'circuit_2qubit_gates': 64, 'circuit_var_gates': 12, 'vqe_variational_parameters': 2}
JW :: -1.1372701683419209
(-0.0988639693354576+0j) [] +
(-0.045322202052874024+0j) [X0 X1 Y2 Y3] +
(0.045322202052874024+0j) [X0 Y1 Y2 X3] +
(0.045322202052874024+0j) [Y0 X1 X2 Y3] +
(-0.045322202052874024+0j) [Y0 Y1 X2 X3] +
(0.1711977490343296+0j) [Z0] +
(0.16862219158920955+0j) [Z0 Z1] +
(0.12054482205301811+0j) [Z0 Z2] +
(0.16586702410589216+0j) [Z0 Z3] +
(0.17119774903432963+0j) [Z1] +
(0.16586702410589216+0j) [Z1 Z2] +
(0.12054482205301811+0j) [Z1 Z3] +
(-0.2227859304041852+0j) [Z2] +
(0.17434844185575676+0j) [Z2 Z3] +
(-0.22278593040418523+0j) [Z3]

{'qubit_hamiltonian_terms': 15, 'circuit_width': 4, 'circuit_depth': 100, 'circuit_2qubit_gates': 64, 'circuit_var_gates': 12, 'vqe_variational_parameters': 2}
{'qubit_hamiltonian_terms': 15, 'circuit_width': 4, 'circuit_depth': 76, 'circuit_2qubit_gates': 46, 'circuit_var_gat

/Users/valentinsenicourt/Desktop/Tangelo_fork/Tangelo/tangelo/toolboxes/qubit_mappings/statevector_mapping.py:149: RuntimeWarning: Symmetry-conserving Bravyi-Kitaev enforces all spin-up followed by all spin-down ordering.
  warnings.warn("Symmetry-conserving Bravyi-Kitaev enforces all spin-up followed by all spin-down ordering.", RuntimeWarning)


### 2b) Compute backends

Tangelo is backend-agnostic. That means that you do not have to rewrite your code if you want to run it on a different simulator or quantum device, and it takes little effort to swap or integrate with a new backend overall.

There's a lot of quantum simulators out there, with different tradeoffs between speed, accuracy, maximal circuit size or ability to simulate noise. Likewise, there's a growing number of quantum devices out there, based on different technologies.

Let's revisit H2 in a bigger basis set, and see what impact the choice of backend has on our experience.

In [8]:
from tangelo.molecule_library import mol_H2_321g

mol = mol_H2_321g

In [9]:
# TODO: Without simulating the algorithm, get an idea of the amount of resources this problem would require
# Assume we are interested in using VQE with the Jordan-Wigner qubit mapping

vqe_options = {"molecule": mol}
vqe_solver = VQESolver(vqe_options)
vqe_solver.build()
print(vqe_solver.get_resources())

{'qubit_hamiltonian_terms': 185, 'circuit_width': 8, 'circuit_depth': 586, 'circuit_2qubit_gates': 432, 'circuit_var_gates': 52, 'vqe_variational_parameters': 9}


Now let's see what happens when you try to run VQE with the `'jw'` mapping but with the following backends. It may take some time depending of your machine, and how optimally these packages were installed (If you've waited more than a few minutes, you can try to revert to H2 in sto-3g basis)

In [10]:
for b in ["qulacs", "cirq", "qiskit"]:
    
    # TODO : Create the dictionary of options to build our VQE object with the desired backends
    backend_options = {"target":b}
    vqe_options = {"molecule": mol, "qubit_mapping": 'jw', "backend_options": backend_options}

    # Start timer
    t_start = time.time()
    
    # Run VQE algorithm
    vqe_solver = VQESolver(vqe_options)
    vqe_solver.build()
    opt_energy = vqe_solver.simulate()
    
    # Stop timer, print elapsed time
    t_stop = time.time()
    print(f"{b:10} :: {t_stop - t_start:3.3f} s \t (energy = {opt_energy})")

qulacs     :: 0.537 s 	 (energy = -1.1478302259877777)
cirq       :: 13.520 s 	 (energy = -1.1478302259878528)
qiskit     :: 88.960 s 	 (energy = -1.1478302259876476)


Roughly, what is the speedup between the best- and poorest-performing backends on your machine ?

## Boss fight: H2O

You want to compute the ground state energy of H2O in sto-3g basis set using VQE: this problem is more challenging than the previous ones. Use all you can find in the source code, documentation and tutorials to give it a go !

- More accuracy is better. Can you reach chemical accuracy (~0.001 Ha) ?
- Research time is important: how much can you reduce time-to-solution with your code ?
- Quantum computers and simulators are limited: how much can you reduce computational requirements

In [11]:
from tangelo.molecule_library import mol_H2O_sto3g
mol = mol_H2O_sto3g

fci_h2o = FCISolver(mol)
opt_e = fci_h2o.simulate()
print(f"H2O FCI energy = {opt_e}")

H2O FCI energy = -75.01277542934066


In [12]:
# TODO: fight !

b = "qulacs"
backend_options = {"target":b}
vqe_options = {"molecule": mol, "qubit_mapping": 'scbk', "backend_options": backend_options}

# Start timer
t_start = time.time()

# Run VQE algorithm
vqe_solver = VQESolver(vqe_options)
vqe_solver.build()
print(vqe_solver.get_resources())
opt_energy = vqe_solver.simulate()

# Stop timer, print elapsed time
t_stop = time.time()
print(f"{b:10} :: {t_stop - t_start:3.3f} s \t (energy = {opt_energy})")

/Users/valentinsenicourt/Desktop/Tangelo_fork/Tangelo/tangelo/toolboxes/qubit_mappings/statevector_mapping.py:149: RuntimeWarning: Symmetry-conserving Bravyi-Kitaev enforces all spin-up followed by all spin-down ordering.
  warnings.warn("Symmetry-conserving Bravyi-Kitaev enforces all spin-up followed by all spin-down ordering.", RuntimeWarning)


{'qubit_hamiltonian_terms': 1086, 'circuit_width': 12, 'circuit_depth': 4337, 'circuit_2qubit_gates': 3800, 'circuit_var_gates': 360, 'vqe_variational_parameters': 65}
qulacs     :: 183.112 s 	 (energy = -75.01263529694484)


In [13]:
from tangelo.algorithms import BuiltInAnsatze as Ansatze

b = "qulacs"
backend_options = {"target":b}
vqe_options = {"molecule": mol, "qubit_mapping": 'scbk', "backend_options": backend_options, 'ansatz': Ansatze.HEA}

# Start timer
t_start = time.time()

# Run VQE algorithm
vqe_solver = VQESolver(vqe_options)
vqe_solver.build()
print(vqe_solver.get_resources())
opt_energy = vqe_solver.simulate()

# Stop timer, print elapsed time
t_stop = time.time()
print(f"{b:10} :: {t_stop - t_start:3.3f} s \t (energy = {opt_energy})")

{'qubit_hamiltonian_terms': 1086, 'circuit_width': 12, 'circuit_depth': 14, 'circuit_2qubit_gates': 22, 'circuit_var_gates': 108, 'vqe_variational_parameters': 108}
qulacs     :: 111.157 s 	 (energy = -74.96311173224746)


# Final words

After all these examples, what observations can you make about the VQE algorithm and what seems to be important in order to apply it successfully to your research ? What were the challenges you've encountered ?

We've only grazed the surface here, but there's more to VQE and its variations. If it is relavant to you, take a look at the [following tutorials](https://github.com/goodchemistryco/Tangelo-Examples/tree/main/examples/variational_methods). Did you know that VQE can also compute [excited states](https://github.com/goodchemistryco/Tangelo-Examples/blob/main/examples/chemistry/excited_states.ipynb)? Keep an eye out, maybe 'watch' the Github repo: the community keeps coming up with new innovations. Maybe we'll owe the next one to you :)

What will you do with Tangelo ?